# Teste para obter base de dados com todos os livros do site
***Site:** [Books to scrape](https://books.toscrape.com/)*

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

from books_ingestion import *
from IPython.display import Image, display

## Rodar pipeline para fazer o webscraping e salvar dados em arquivo .csv

In [2]:
%%time
populate_books(url_books)

Início da execução                                                    : 09/01/2026 15:02:39
Obtendo o dicionário das categorias e respectivos links               : 09/01/2026 15:02:39

Iterando as categorias para obter os dados dos livros                 : 09/01/2026 15:02:40


100%|██████████| 50/50 [01:05<00:00,  1.31s/it]

Término do loop                                                       : 09/01/2026 15:03:46

Tamanho da base de dados: 1000
Base de dados salva                                                   : 09/01/2026 15:03:46

CPU times: total: 4.28 s
Wall time: 1min 10s


## Testar base de dados salva

In [3]:
base_livros = pd.read_csv('../../data/base_livros.csv')

print(f"Tamanho da base: {base_livros.shape[0]}")
base_livros.head()

Tamanho da base: 999


,id,title,price,rating,availability,category,image
0,1,It's Only the Himalayas,45.17,2,In stock,Travel,https://books.toscrape.com/media/cache/27/a5/2...
1,2,Full Moon over Noah’s Ark: An Odyssey to Mount...,49.43,4,In stock,Travel,https://books.toscrape.com/media/cache/57/77/5...
2,3,See America: A Celebration of Our National Par...,48.87,3,In stock,Travel,https://books.toscrape.com/media/cache/9a/7e/9...
3,4,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,In stock,Travel,https://books.toscrape.com/media/cache/d5/bf/d...
4,5,Under the Tuscan Sun,37.33,3,In stock,Travel,https://books.toscrape.com/media/cache/98/c2/9...


In [6]:
display(Image(url = base_livros.image[222]))